# Функции

Функции и типы данных -- основные единицы переиспользования кода.

Хотя интерпретируемые языки *позволяют* писать программу "одним куском" без разделения, лучше так не делать и применять **декомпозицию на функции**. Преимущества этого:
* проще читать
* проще понимать
* проще отлаживать
* проще компоновать

## Функция с точки зрения математики

В математике функция -- однозначное отображение из области определения во множество значений.

Т.е. у функции есть *аргумент* -- один или в виде кортежа, -- которому в соответствие ставится *значение*, если он является допустимым.

Очевидное свойство -- одному аргументу функции соответствует единственное значение.

## Функция с точки зрения программирования

В программировании функция -- это *подпрограмма*, в идеале применяемая в первую очередь для получения значения на основе своих аргументов.

Поведение функции как подпрограммы может определяться неким *состоянием* программы, из-за чего понимание функции отличается от математического -- подпрограмма при вызове с теми же аргументами может давать разный результат.

Функции, которые при применении к одному и тому же значению дают один и тот же результат, называют *чистыми*.

Пример функции, не являющейся чистой:

In [1]:
from random import random

In [2]:
random()

0.2943392971295712

In [3]:
random()

0.23557876079831386

In [4]:
random() == random()

False

Обычно функции, не имеющие никаких аргументов -- не чистые, поскольку чистые можно было бы заменить просто на значения. Исключениями будут разве что конструкторы типов, которые возвращают, конечно, каждый раз *разные* объекты (расположенные в разных ячейках памяти), но имеющие одинаковые значения.

Пример чистой функции:

In [5]:
def square(x):
    return x * x

In [6]:
square(5) == square(5)

True

Значение чистой функции определяется значениями её аргументов, определениями других входящих в её тело функций, константными значениями **и ничем более**. Т.е. чистая функция внутри себя может пользоваться только другими чистыми функциями.

Про функцию, не являющуюся чистой, говорят, что она производит *побочный эффект*. К побочным эффектам относятся
* чтение из потока ввода (меняется положение "курсора" в потоке)
* запись в поток вывода
* изменение состояния некоторого программного объекта
* изменение состояния переданных аргументов

т.е. всё, что позволяет программисту как-то отличить состояния "до" вызова функции и "после" (на уровне компьютерного железа, очевидно, любое выполнение подпрограммы приведёт к каким-то изменениям -- например, изменятся значения регистров, -- но из программы они не обязаны быть напрямую доступны).

Опасность побочных эффектов -- в том, что они, подобно квантовой запутанности, вызывают "spooky action at a distance". В результате ошибка может быть *не обязательно в тексте функции, но и в положении вызова этой функции относительно остальных*.

Пример того, что функции с побочными эффектами должны вызываться в строго определённом порядке:

In [7]:
from io import StringIO
import numpy as np
import scipy
import scipy.linalg

def read_header(file):
    first_string = file.readline()
    return first_string.split()

def read_data(file, colnames=None):
    if colnames is not None:
        format = {"names": colnames, "formats": ["f4" for _ in colnames]}
        return np.loadtxt(file, dtype=format)
    else:
        return np.loadtxt(file)

def read_table(file):
    colnames = read_header(file)
    return read_data(file)

In [8]:
io = StringIO("""Step Temp Press Pxy 
       0            1   0.55911568   0.10211457 
     100   0.97820637   0.63393299  0.069052478 
     200    1.0119581   0.47629965   0.12195751 
     300   0.99689111     0.545694  0.010526807 
     400   0.99459354    0.5391264   0.07315686 
     500   0.95087088   0.77035112  0.063373937 
     600    1.0213663   0.37514456   0.16874407 
     700    1.0177222    0.3938397 -0.062810141 
     800     1.007198   0.51861816  -0.20937204 
     900    1.0006034   0.62299635   0.12676103 
    1000    1.0174942   0.30372804   0.11736125 
""")

read_table(io)

array([[ 0.0000000e+00,  1.0000000e+00,  5.5911568e-01,  1.0211457e-01],
       [ 1.0000000e+02,  9.7820637e-01,  6.3393299e-01,  6.9052478e-02],
       [ 2.0000000e+02,  1.0119581e+00,  4.7629965e-01,  1.2195751e-01],
       [ 3.0000000e+02,  9.9689111e-01,  5.4569400e-01,  1.0526807e-02],
       [ 4.0000000e+02,  9.9459354e-01,  5.3912640e-01,  7.3156860e-02],
       [ 5.0000000e+02,  9.5087088e-01,  7.7035112e-01,  6.3373937e-02],
       [ 6.0000000e+02,  1.0213663e+00,  3.7514456e-01,  1.6874407e-01],
       [ 7.0000000e+02,  1.0177222e+00,  3.9383970e-01, -6.2810141e-02],
       [ 8.0000000e+02,  1.0071980e+00,  5.1861816e-01, -2.0937204e-01],
       [ 9.0000000e+02,  1.0006034e+00,  6.2299635e-01,  1.2676103e-01],
       [ 1.0000000e+03,  1.0174942e+00,  3.0372804e-01,  1.1736125e-01]])

Очевидно, что для чтения данных из лога LAMMPS в показанном выше формате внутри функции `read_table` вызов `read_header(file)` должен стоять перед `read_data(file)`, причём ровно в единственном экземпляре.

Методически можно отметить также осмысленность выделения операции, которая в данном случае просто читает одну строчку из файла и разбивает её на слова, в функцию `read_header`. При чтении текста основной функции `read_table` сразу становится понятно, что ожидается некий вход, содержащий заголовок. К тому же, от заголовка может быть польза -- `read_table` можно переписать как

```python
def read_table(file):
    colnames = read_header(file)
    return read_data(file, colnames)
```

Тогда считанные имена столбцов будут включены в таблицу.

Особняком стоят побочные эффекты, связанные с изменением состояния переданных в функцию аргументов. Довольно часто, если из функции нужно получить массив или другой контейнер, результат переписывает содержимое одного или нескольких из переданных аргументов. Такие аргументы называют *выходными*.

В простых скриптах привлекательно выглядит вариант, когда вместо выходных аргументов переписывается какая-нибудь глобальная переменная. **Так лучше не делать**. Ситуация, когда все данные, от которых зависит результат вычисления функции, она получает из аргументов, и все возможные изменения локализованы там же, -- гораздо удобнее для отладки, модификации и чтения другими людьми.

In [9]:
def normalize_rows(array):
    for r in range(array.shape[0]):
        array[r,:] /= np.linalg.norm(array[r, :])
    return array

In [10]:
matr = np.random.rand(5, 5)
matr

array([[0.74192605, 0.77667331, 0.75119021, 0.58291164, 0.63467427],
       [0.79412949, 0.66812904, 0.1100631 , 0.63064272, 0.80257086],
       [0.67044436, 0.00970945, 0.96063284, 0.76766735, 0.82242292],
       [0.11471715, 0.90602322, 0.62968084, 0.61708868, 0.49958341],
       [0.8295284 , 0.30274072, 0.3042647 , 0.15145893, 0.53280984]])

In [11]:
normalize_rows(matr)
matr

array([[0.47298096, 0.49513248, 0.4788869 , 0.37160861, 0.4046075 ],
       [0.54400315, 0.45768896, 0.07539661, 0.43200968, 0.54978575],
       [0.4127787 , 0.00597791, 0.59144173, 0.47263688, 0.50634875],
       [0.08409395, 0.66416456, 0.46159049, 0.45235975, 0.36622196],
       [0.76391071, 0.2787932 , 0.28019663, 0.13947817, 0.4906633 ]])

# Области видимости переменных (идентификаторов)

Ещё одна причина для функциональной декомпозиции сложных задач -- не запутаться при написании большой программы или функции. Одним из источников ошибок, особенно в языках с динамической типизацией, может быть случайное нецелевое использование имени, которое уже применено для других целей.

Как иллюстрацию можно привести такой (довольно надуманный) пример для случая, когда все вычисления сделаны в toplevel скрипта без декомпозиции на функции (или в одной большой функции, что по сути одно и то же):

```python
a = []
# какая-то работа с `a` и другими переменными

# забыли, что `a` уже где-то использовалось
if some_condition:
    a = 5
elif another_condition:
    a = 0

print(x + a * y)
```

Опасность состоит в том, что в условном выражении могут быть прописаны условия, не покрывающие все случаи. И тогда, если вдруг случится что-либо не подпадающее ни под `some_condition`, ни под `another_condition`, то в `a` вместо числа окажется массив.

Ограничение *области видимости* имён -- т.е. части программы, где одно и то же имя означает одну и ту же сущность -- вместе с разбиением на подпрограммы сглаживает эту проблему.

Вспомним, что идентификатор -- это любая допустимая строка в тексте программы, не являющаяся
* литералом (`1`, `"zzz"`, `True`, `None`)
* ключевым словом (`def`, `for` и т.п.)
* зарезервированным символом оператора (`+`, `()`, `[]`, `:` и т.п.)
* частью комментария

Идентификаторы используются для именования переменных, функций, классов и т.п. Ограничение области видимости означает, что смысл, вкладываемый в одно и то же имя в разных частях программы, может быть разным без возникновения конфликта, затирания старых значений и т.п.

Рассмотрим пример:

```python
array = numpy.array([[1.0, 2.0], [3.0, 4.0]])

def normalize_rows(array):
    array = array.copy()
    for r in range(array.shape[0]):
        array[r,:] /= numpy.linalg.norm(array[r, :])

    return array

new_array = normalize_rows(array)
```

Здесь `array` используется:
* как имя переменной
* как имя формального аргумента функции `normalize_rows`
* как имя функции из модуля `numpy`

При выполнении этого кода оказывается, что хотя в теле функции `normalize_rows` выполнено присваивание `array = array.copy()`, `array` после выполнении функции не меняется.

In [12]:
array = np.array([[1.0, 2.0], [3.0, 4.0]])

def normalized_rows(array):
    array = array.copy()
    for r in range(array.shape[0]):
        array[r,:] /= np.linalg.norm(array[r,:])
    print("`array` =\n{}".format(array))
    return array

print("global `array` before call:\n{}\n".format(array))
new_array = normalized_rows(array)

print("\nglobal `array` after call:\n{}".format(array))

global `array` before call:
[[1. 2.]
 [3. 4.]]

`array` =
[[0.4472136  0.89442719]
 [0.6        0.8       ]]

global `array` after call:
[[1. 2.]
 [3. 4.]]


 Причина в том, что в Python имеются разные области видимости.

Тело любой функции составляет *локальную область видимости*. В неё входят
* имена формальных аргументов
* любые имена, которые в теле функции присутствуют слева от знака присваивания
* имена переменных цикла `for`
* имена функций, определяемых в теле функции

Поскольку внутри `normalize_rows` имя `array` используется для формального аргумента -- оно является *связанным* в рамках этой функции, и любые присваивания для этого имени не отражаются на переменных с тем же именем вне функции. Более того, существование этого имени вне функции никоим образом не требуется для её работоспособности.

С другой стороны, имя `range` внутри функции является *свободным*. Это значит, что его значение определяется наличием смысла для этого имени вне функции. В данном случае, очевидно, имеется в виду стандартная функция `range`. Но язык не запрещает написать программу так:

```python
def range(x):
    return None

def normalize_rows(array):
    array = array.copy()
    for r in range(array.shape[0]):
        array[r,:] /= numpy.linalg.norm(array[r, :])
    return array
```

Тогда `normalize_rows`, естественно, будет завершаться с ошибкой. Ещё более болезненно то, что даже не важно, "отравленное" определение `range` будет стоять до или после определения `normalize_rows`. Все вызовы `normalize_rows`, расположенные в программе после нового определения, интерпретатор будет пытаться проделать именно с ним. Хотя в принципе в рамках функции можно создать переменную, имя которой экранирует имя встроенной функции (пусть это будет `range`), и при этом по-прежнему иметь возможность обращаться ко встроенной функции через `__builtins__.range`.

Для видимости переменных в Python применяются два основных правила:
1. Внутри тела функции имена её формальных аргументов и всех переменных, в которые осуществляется присваивание, считаются *связанными* или *локальными*. Если вне функции объявлены идентификаторы с таким же именем, они *экранируются* локальным именем.
2. Все остальные имена берутся из внешней ("объемлющей") области видимости, причём поиск идёт по тем же принципам -- значение берётся из наиболее вложенной области видимости, в которой его связывание встречается.

Внутри тела функции область видимости плоская -- т.е. внутри блочных конструкций (ветвлений, циклов) новые области видимости не создаются. Это же относится к переменным циклов: в конструкции

```python
for x in collection:
    do_something(x)
```
последнее значение `x` доступно после выхода из цикла. Также, если перед циклом было присваивание переменной `x` какого-то значения, при входе в цикл оно потеряется.

Таким образом, внутри функции доступны все имена, определённые вне её, но только "на чтение". Если внутри функции есть присваивание в некоторую переменную, то интерпретатор считает такую переменную локальной.

In [13]:
greeting = "Hello"

def german_greet():
    greeting = "Hallo"
    print(greeting)
    
print("greeting = {}".format(greeting))

german_greet()

print("greeting = {}".format(greeting))

greeting = Hello
Hallo
greeting = Hello


Для указания, что выражение `x = y` должно не создавать локальную переменную `x`, а изменить или создать глобальную переменную `x`, в Python нужно сначала пометить имя `x` внутри функции как глобальное: `global x`.

Также, что актуально для вложенных функций, выражение `x = y` в теле вложенной функции означает создание локальной переменной вне зависимости от того, есть во внешней функции связывание для `x` или нет:

In [14]:
def enclosing1(x):
    y = x * 2
    def inner1(z):
        y = z**2
        print("Inner1 y = {}".format(y))
        return y
        
    def inner2(z):
        yy = z**2
        print("Inner2 y = {}".format(y))
        return yy
    
    inner1(x)
    inner2(x)
    print("Enclosing y = {}".format(y))
    return y

enclosing1(5)

Inner1 y = 25
Inner2 y = 10
Enclosing y = 10


10

Для изменения переменной в ближайшей из объемлющих областей видимости используется ключевое слово `nonlocal`:

In [15]:
def enclosing2(x):
    y = x * 2
    def inner1(z):
        nonlocal y
        y= z**2
        print("Inner1 y = {}".format(y))
        return y
        
    def inner2(z):
        yy = z**2
        print("Inner2 y = {}".format(y))
        return yy
    
    inner1(x)
    inner2(x)
    print("Enclosing y = {}".format(y))
    return y

enclosing2(5)

Inner1 y = 25
Inner2 y = 25
Enclosing y = 25


25

### Замыкания

Наибольшую важность идея областей видимости переменных имеют в случае, когда одна функция возвращает другую как значение. В этом случае внутренняя функция и переменные, которые она использует из внешней, образуют так называемое *лексическое замыкание*.

Как пример -- генератор квадратных трёхчленов

```python
def make_quadratic(a, b, c):
    def quatratic(x):
        return a * x**2 + b * x + c
    return quadratic
```

Тонкость здесь состоит в том, что изменение значений включенных в замыкание переменных влияет на поведение самих функций -- другими словами, свободные идентификаторы в замыкании связываются со ссылками на значения этих идентификаторов во внешней области видимости. В этом смысле "обычные" функции также являются замыканиями, для которых единственная внешняя область видимости -- глобальная.

Следующий пример демонстрирует, что значение захваченного идентификатора в замыкании фиксируется не в точке определения внутренней функции, а связано с областью видимости внешней функции, которая "продолжает жить" только в связке с замыканием:

In [16]:
def multiplications(n):
    closures = [None for _ in range(n)]
    
    for k in range(n):
        def multiply_by_k(x):
            return k * x
            
        closures[k] = multiply_by_k
    
    return closures

Можно было бы подумать, что в массиве `multiplications(n)` лежат функции, умножающие аргументы на $0, 1, \dots, n$. Однако, поскольку область видимости переменной `k` -- *вся функция* `multiplications`, во всех замыканиях `k` обозначает один и тот же объект: 

In [17]:
[closure(1) for closure in multiplications(3)]

[2, 2, 2]

Чтобы переменные `k` в каждом замыкании имели разное значение, нужно создать ещё одну вложенную область видимости, в которой зафиксировать `k`. Такая область видимости должна создаваться заново на каждой итерации цикла.

В Python единственный способ создания таких областей видимости -- вызвать в цикле функцию, в которую `k` будет передаваться как аргумент:

In [18]:
def multiplications_fix(n):
    closures = [None for _ in range(n)]
    
    def gen_multiplicator(mult):
        def multiplicator(x):
            return mult * x
        return multiplicator

    for k in range(n):
        closures[k] = gen_multiplicator(k)
    
    return closures

In [19]:
[closure(1) for closure in multiplications_fix(3)]

[0, 1, 2]

Этот пример показывает, что передача данных в функции через внешние переменные, не являющиеся аргументами, может запутывать программу.

Во втором примере посмотрим поведение двух замыканий, в которых переменная из внешней области видимости обозначена как `nonlocal` и может изменяться:

In [20]:
def make_counter(init=0):
    c = init
    def count():
        nonlocal c
        c += 1
        return c
    
    def reset_count(init=init):
        nonlocal c
        c = init
        
    def get_count():
        return c
    
    return get_count, count, reset_count

In [21]:
get_count, count, reset_count = make_counter()

In [22]:
count()

1

In [23]:
count()

2

In [24]:
reset_count()

In [25]:
count()

1

In [26]:
get_count()

1

Как видно, изменение связанного с `c` значения, которое делается в `count` и `reset_count`, распространяется на всю область видимости, т.е. и на то, что получает `get_count`.

Такое поведение, хотя и немного странное, похоже на поведение объектов и их методов. Это лучше видно, если замыкания возвращать как именованный кортеж:

In [27]:
from collections import namedtuple

def make_person(name, age):
    def get_name():
        return name
    
    def get_age():
        return age
    
    def birthday():
        nonlocal age
        age += 1
        print("{}'s age is now {}".format(name, age))
        
    person_t = namedtuple('Person', ['name', 'age', 'birthday'])
    return person_t(get_name, get_age, birthday)

In [28]:
JohnDoe = make_person("John Doe", 18)

In [29]:
JohnDoe.name()

'John Doe'

In [30]:
JohnDoe.age()

18

In [31]:
JohnDoe.birthday()

John Doe's age is now 19


In [32]:
JohnDoe.age()

19

Такая форма показывает, что набор функций, которые оперируют с одним и тем же изменяемым состоянием неявно (т.е. не через свои аргументы), нужно рассматривать как единый объект.

Второй пример ещё раз предостерегает от использования изменяемых глобальных переменных -- при их наличии вызов любой функции может изменить поведение другой, хотя в этом может не быть явного намерения.